# GRU model

We start by importing the necessary libraries

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from core.util.plot_predictions import plot_predictions
from core.util.get_datasets import get_park_datasets
from core.util.trefor_dataset import TreforData
from core.models import GRU
from core.models.model_training import train_model, test_model
from core.util.hyperparameter_configuration import get_hyperparameter_configuration
from core.util.early_stop import EarlyStop

## Setup
This section contains the setup for converting our data to something PyTorch can understand.
### Set hyperparameters for the model

In [ ]:
# To be modified depending on gridsearch result
batch_size = 128
learning_rate = 0.001
num_layers = 1
lookback = 36

# Extract hyperparameters configuration that will not be tuned upon
hidden_size, EPOCHS, horizon, lookback, loss_function, dropout_rate, folds = (
    get_hyperparameter_configuration()
)

### Get the park datasets and convert them to torch tensors

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test, indicies = get_park_datasets(
    lookback=lookback, horizon=horizon, folds=folds
)

### If the host has CUDA, it will use the GPU for computation

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

### Convert to specific Datasets for conversion to Dataloaders

In [ ]:
train_dataset = TreforData(x_train, y_train, device)
val_dataset = TreforData(x_val, y_val, device)
test_dataset = TreforData(x_test, y_test, device)

### Convert to data to dataloaders
The DataLoader class helps us split the dataset into batches, and can also shuffle the data in between epochs if desired.

In [ ]:
training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testing_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Creation of our simple GRU model
The implementation consist of three layers, defined in the `forward` method.
1. GRU
2. LeakyReLU
3. Linear

In [ ]:
model = GRU(
    input_size=1,
    hidden_size=10,
    num_layers=num_layers,
    dropout_rate=dropout_rate,
    horizon=horizon,
    lookback=lookback,
)
model.to(device)

### Training loop
Iterate the epochs, training and validating the model.

We save the model, in case it is better than any of the previous we have seen.

In [ ]:
plot_train_loss, plot_val_loss, best_model = train_model(
    epochs=EPOCHS,
    model=model,
    loss_function=loss_function,
    training_loader=training_loader,
    validation_loader=validation_loader,
    learning_rate=learning_rate,
    early_stopper=EarlyStop(5, 0.05),
)

### Use the best model to make predictions on the test set
This is very similar to the validation predictions above, only that this is run just once.

In [ ]:
t_loss, predicted = test_model(
    best_model=best_model, loss_function=loss_function, testing_loader=testing_loader
)
print(f"Avg loss: {t_loss:>8f} \n")

## Plotting
Show some nice output :)
### Training- and validation loss

In [ ]:
plt.clf()
plt.plot(plot_train_loss, label="Training Loss")
plt.plot(plot_val_loss, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Predictions- and actual consumption

In [ ]:
plot_predictions(500, 514, y_test, predicted)